In [ ]:
# coding=utf-8
import tensorflow as tf
import numpy as np

# 查阅文档
print(dir(tf.dtypes))
print(dir(tf.nn.relu))
print(tf.nn.relu.__name__)
print(help(tf.ones))

# tensor 创建:
# (1) tf.constant() & tf.convert_to_tensor()
x = tf.constant(range(5), dtype=tf.int32) # <tf.Tensor: id=2, shape=(5,), dtype=int32, numpy=array([0, 1, 2, 3, 4])>
x = tf.constant(list([1,2,3]))
x = tf.constant(np.random.random((2,3)))
x = tf.convert_to_tensor(np.random.random(6).reshape(2,3))

# (2) 常见矩阵
x = tf.zeros((2,3,4))
x = tf.ones((2,3))
x = tf.ones_like(x, dtype=tf.int16)
x = tf.eye(3)

# (3) fill 重复填充值
x = tf.fill([2,2], value=5)

# (4) 随机 tf.random 模块
x = tf.random.uniform(shape=(2,3), minval=0, maxval=100, seed=None) # 均匀分布
x = tf.random.normal(shape=(2,3), mean=x, stddev=10, seed=None) # 正太分布
x = tf.random.truncated_normal(shape=(2,3), mean=x, stddev=1) # 截取2个标准差内的正太分布
x = tf.random.shuffle(x, seed=None)

In [ ]:
# tensor属性
x = tf.constant(range(3))
print(x.numpy, x.shape, tf.rank(x), x.ndim, x.dtype, len(x))

# tensor 维度说明： tf默认［N,H,W,C］（ＣＰＵ上预测用）, 读图 [N,C,H,W]（ＧＰＵ上训练用）
#  因为早期tf用于cpu推导，所以默认的是 ＮＨＷＣ，单张图像［ＣＨＷ］

# 转Numpy
np.array(x) # 或者 x.numpy

# 类型转换
tf.cast(x, tf.int32)

# 存储转换
with tf.device("CPU:0"):
    x = tf.constant(range(3))
    print(x.device)
    y = x.gpu()
    print(y.device)


In [ ]:
# 运算
with tf.device("CPU:0"):
    x = tf.constant(range(1,7))
    x = tf.reshape(x, (2,3))
    y = tf.ones((2,3)) * 2.0
    x = tf.cast(x, y.dtype)    # 类型转换，否则int+float非法
    # print(x,y)
    
    # (1) 元素运算
    x * y
    x / y
    x + y

    # (2) 矩阵乘法
    tf.matmul(x, tf.transpose(y))
    
    # (3) 矩阵比较
    tf.equal(x, y)
    tf.not_equal(x, y)
    tf.less(x, y)
    tf.less_equal(x, y)
    tf.greater(x, y)
    tf.greater_equal(x, y)

    tf.math.is_nan(x)
    tf.math.is_finite(x)
    tf.math.is_nan(x)

    # (4) 矩阵拼接 [channel, row, col]
    z = tf.concat([tf.reshape(x,(1,2,3)), tf.reshape(y,(1,2,3))], axis=0) # 模拟通道串联
    # print(z)

    # (5) 矩阵 reduce: 维度合并
    print(tf.reduce_max(z, axis=0)) # 在 axis 上求最大值, axis是要减少合并的维度
    tf.reduce_min(z, axis=0)
    tf.reduce_mean(z, axis=0)
    tf.reduce_sum(z, axis=0)  # 还有亦或运算：tf.reduce_all, tf.reduce_any
   
    # (6)* 广播机制：维度不对的两个tensor运算，可能被自动填充后计算
    A = tf.reshape(tf.constant(range(3)), (3,1))
    B = tf.reshape(tf.constant(range(2)), (1,2))
    # A, B, A + B

    # (7) 张量分割
    x = tf.reshape(tf.constant(range(1,9)),(1,2,4))
    y = tf.split(x,num_or_size_splits=2, axis=2)
    # print(x, y)

    # (8) map_fn() 映射函数
    # 要将函数操作应用于 SparseTensor 的非零元素,建议使用以下方法之一.首先,如果函数可以表示为 TensorFlow ops,请使用：
    # result = SparseTensor(input.indices, fn(input.values), input.dense_shape)
    # 但是,如果该函数不能作为 TensorFlow op 表示,则使用：
    # result = SparseTensor(input.indices, map_fn(fn, input.values), input.dense_shape)
    with tf.device("CPU:0"):
        elems = np.array([1, 2, 3], dtype=np.int64)
        alternates = tf.map_fn(lambda x: (x, -x), elems, dtype=(tf.int64, tf.int64))
        print('map_fn: ', alternates)

        x = tf.constant(np.random.random(6).reshape(2,3))
        y = tf.map_fn(lambda a: a+1, x)
        print(x, y)

    # (9) 条件函数
    with tf.device("CPU:0"):
        x = tf.constant(np.random.random(6).reshape(2,3))
        y = tf.constant(np.random.random(6).reshape(2,3))
        # tf.cond(cond, true_val, false_val) 中，函数一定都是 callable 对象
        z = tf.cond(lambda: tf.less(x, y), lambda: tf.add(x,y), lambda: tf.square(x))
        def f2(): return y**2
        w = tf.cond(lambda: tf.less(x, y), lambda: tf.add(x,y), lambda x: x**2,)
        print(z)

    # (10) Where 函数
    with tf.device("CPU:0"):
        x = tf.constant(np.random.random(6).reshape(2,3))
        y = tf.constant(np.random.random(6).reshape(2,3))
        z = tf.where(x<y, x+y, y**2)
        print(z)

    # (11) 切片函数 tf.slice
    # (12) 排序函数 tf.sort(values, axis=-1, direction='ASCENDING', name=None)
   


   

In [ ]:
# 变量创建
x = tf.Variable(3)
x = tf.Variable(np.random.random(3))
x = tf.Variable(tf.constant(3))
x = tf.Variable(tf.random.normal(shape=(2,3), mean=0, stddev=1))
print(x)

# 变量赋值
x[1,:].assign([1,1,1])
print(x)

# 只有 assign系列，+=系列 不会新开内存
with tf.device("CPU:0"):
    x = tf.Variable(3, dtype=tf.float16)
    print(id(x), x)
    x.assign(0)
    print(id(x), x)
    x.assign_add(1)
    print(id(x), x)
    x = 2
    print(id(x), x)
    x = x + 1
    print(id(x), x)



In [ ]:
# 查阅文档

print(dir(tf.dtypes))

print(help(tf.ones))












In [ ]:
import cv2

img = cv2.imread('data/tj.png')
print(img.shape)

def isw(pix):
    return pix[0]>254 and pix[1]>254 and pix[2]>254

# 571, 750, 3

for i in range(img.shape[0]):
    for j in range(img.shape[1]):
        if isw(img[i][j]):
            img[i][j] = [250,180,0]
        else:
            img[i][j] = [100,70,10]

cv2.imshow('',img)
cv2.imwrite('data/bkg.jpg', img)
cv2.waitKey()
            